# BASIC AGENT LANGGRAPH

In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=2) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [5]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai) # add the LLM node
        graph.add_node("action", self.take_action) # add the action node
        graph.add_conditional_edges( 
            "llm",
            self.exists_action,
            {True: "action", False: END}
        ) # add the conditional edge, if the LLM returns a tool call, then take the action node, otherwise, end the graph
        graph.add_edge("action", "llm") # add the edge from the action node to the LLM node, an edge is a connection between nodes
        graph.set_entry_point("llm") # set the entry point of the graph to the LLM node, this means that the graph will start at the LLM node
        self.graph = graph.compile() # compile the graph into a callable function
        self.tools = {t.name: t for t in tools} # bind the tools to the agent
        self.model = model.bind_tools(tools) # bind the model to the tools

    def exists_action(self, state: AgentState):
        """
        Check if the LLM has returned an action call in the last message. 
        params:
            state: The current state of the agent.
        returns:
            True if the LLM has returned an action call, False otherwise.
        """
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        """
        Call the OpenAI model to get a response to the current message.
        params:
            state: The current state of the agent.
        returns:
            The response from the OpenAI model.
        """
        
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        """
        Take an action based on the tool calls in the last message.
        params:
            state: The current state of the agent.
        returns:
            The response from the OpenAI model.
        """
        
        
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [6]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [7]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_O2hoBddgSzfm8vJlHGdSXsEI'}
Back to the model!


In [8]:
result

{'messages': [HumanMessage(content='What is the weather in sf?'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_O2hoBddgSzfm8vJlHGdSXsEI', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6edba178-6087-4922-898d-65e528add44d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_O2hoBddgSzfm8vJlHGdSXsEI'}], usage_metadata={'input_tokens': 153, 'output_tokens': 21, 'total_tokens': 174}),
  ToolMessage(content='[{\'url\': \'http://www.sfchronicle.com/weather-forecast/article/thunderstorms-lightning-tropical-california-19530762.php\', \'content\': \'Temperatures will be seasonably cool, in the upper 50s in 

In [9]:
result['messages'][-1].content

'The weather in San Francisco is seasonably cool with temperatures in the upper 50s in the Sunset and Richmond districts, low 60s in the Marina and Panhandle, and mid-60s in SoMa, downtown, and the Mission. Highs will be near 60 at the coast and in the 70s to low 80s near the bay shoreline and 80s to low 90s inland. There is a prediction of a few weak thunderstorms across the Bay Area and Central Valley with stronger showers and thunderstorms over the Sierra Nevada.'

In [10]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

query = "Who won austria f1 race 2024? Where there any accident? \
get the drivers and total wins in f1 career. \
    " 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Austria F1 race 2024 winner'}, 'id': 'call_NPXFqcn10c2AqHVW2cPL4Dbx'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Austria F1 race 2024 accidents'}, 'id': 'call_EIla6SDuOc3Jxsff89Wp7DMA'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'George Russell total F1 wins in career'}, 'id': 'call_FuPJ74O4rOG1Bp24D6MrMpcs'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Max Verstappen total F1 wins in career'}, 'id': 'call_5cRcxacM4qPQ8HLW4BeMVE3a'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Lando Norris total F1 wins in career'}, 'id': 'call_586102oItz8eUvJrm3aEpip5'}
Back to the model!


In [11]:
print(result['messages'][-1].content)

### Austria F1 Race 2024 Results and Incidents

**Winner of the Austria F1 Race 2024:**
- **George Russell** was the unlikely winner of the 2024 Austrian Grand Prix. This victory marked his second career F1 race win. The win was a result of a late collision between race leaders Max Verstappen and Lando Norris.

**Accidents:**
- There was a significant incident involving **Max Verstappen** and **Lando Norris**. The two drivers collided while battling for the lead, which led to their retirement from the race.

### Total Wins in F1 Career

**1. George Russell:**
- **Total Wins:** 2 career F1 race wins.

**2. Max Verstappen:**
- **Total Wins:** 60 career F1 race wins. His 60th win was recorded at the Canadian Grand Prix in 2024.

**3. Lando Norris:**
- **Total Wins:** 1 career F1 race win. He secured his maiden win in Miami in 2024.

### Summary
- George Russell clinched victory at the Austria F1 race in 2024 due to a collision between Max Verstappen and Lando Norris.
- Max Verstappen has 